In [34]:
import fxcmpy
import time
import datetime as dt
from datetime import datetime
from pyti.simple_moving_average import simple_moving_average as sma
from dateutil.relativedelta import relativedelta

In [93]:
token = '705d276b3730e9b53952cce3c135552830994573'
symbol = 'EUR/USD'
timeframe = "m5"        # (m1,m5,m15,m30,H1,H2,H3,H4,H6,H8,D1,W1,M1)
fast_sma_periods = 10
slow_sma_periods = 20
close_on_opposing_signal = True
amount = 10
stop = -10
limit = 30

In [94]:
pricedata = None
numberofcandles = 20

In [4]:
con = fxcmpy.fxcmpy(access_token=token, log_level="error", server='demo')

In [9]:
def Prepare():
    print("Requesting Initial Price Data...")
    pricedata = con.get_candles(symbol, period=timeframe, number=numberofcandles)
    print("Initial Price Data Received...")
    return pricedata

def StrategyHeartBeat():
    while True:
        currenttime = dt.datetime.now()
#       if timeframe == "m5" and currenttime.second == 0 and currenttime.minute % 5 == 0 and GetLatestPriceData(): 
        Prepare()
        time.sleep(60)
        

In [10]:
pdata = Prepare()

Requesting Initial Price Data...
Initial Price Data Received...


In [83]:
pdata.tail()

,bidopen,bidclose,bidhigh,bidlow,askopen,askclose,askhigh,asklow,tickqty
date,,,,,,,,,
2019-07-09 07:35:00,1.12116,1.12107,1.12127,1.12105,1.12128,1.12120,1.12140,1.12117,835
2019-07-09 07:40:00,1.12107,1.12093,1.12109,1.12082,1.12120,1.12105,1.12121,1.12093,701
2019-07-09 07:45:00,1.12094,1.12093,1.12103,1.12083,1.12106,1.12105,1.12115,1.12095,727
2019-07-09 07:50:00,1.12093,1.12063,1.12101,1.12061,1.12105,1.12076,1.12113,1.12073,758
2019-07-09 07:55:00,1.12063,1.12071,1.12073,1.12044,1.12076,1.12084,1.12085,1.12056,974


In [76]:
def check_updated(dfold, dfnew):
    last_time_old = dfold.index[dfold.shape[0]-1]
    last_time_new = dfnew.index[dfnew.shape[0]-1]
    if last_time_new>last_time_old:
        return True

In [89]:
def get_latest_data():
    print("Getting initial data...")
    pricedata = Prepare()
    print("initial price data")
    print(pricedata.tail())
    while True:
        last_time = pricedata.index[pricedata.shape[0]-1]
        curr_time = dt.datetime.now()
        if curr_time>last_time:
            
            print('requesting new data...')
            newprice_data = con.get_candles(symbol, period=timeframe, number=numberofcandles)
        if check_updated(dfold=pricedata, dfnew=newprice_data):
            pricedata = newprice_data
            print("new price data received...")
            print(pricedata.tail())
            Update()
        time.sleep(30)

In [95]:
def enter(BuySell):
    direction = True;
    if BuySell == "S":
        direction = False;
    try:
        opentrade = con.open_trade(symbol=symbol, is_buy=direction, amount=amount, time_in_force='GTC', order_type='AtMarket',is_in_pips=True,limit=limit, stop=stop)
    except Exception as e:
        print("  Error Opening . {}".format(str(e)))
    else:
        print(" Opened Successfully ")

# This function closes all positions that are in the direction BuySell, "B" = Close All Buy Positions, "S" = Close All Sell Positions, uses symbol
def exit(BuySell=None):
    openpositions = con.get_open_positions(kind='list')
    isbuy = True
    if BuySell == "S":
        isbuy = False
    for position in openpositions:
        if position['currency'] == symbol:
            if BuySell is None or position['isBuy'] == isbuy:
                print("  Closing tradeID: " + position['tradeId'])
                try:
                    closetrade = con.close_trade(trade_id=position['tradeId'], amount=position['amountK'])
                except Exception as e:
                    print("  Error Closing {}".format(str(e)))
                else:
                    print(" Closed Successfully ")

In [96]:
def Update():
    print(str(dt.datetime.now()) + "	 " + timeframe + " Bar Closed - Running Update Function...")

    # Calculate Indicators
#     iFastSMA = sma(pricedata['bidclose'], fast_sma_periods)
#     iSlowSMA = sma(pricedata['bidclose'], slow_sma_periods)pricedata
    pricedata['dir'] = pricedata.apply(lambda x: 1 if x['bidopen']<x['bidclose'] else -1 if x['bidopen']>x['bidclose'] else 0, axis=1)
    # Print Price/Indicator
    # TRADING LOGIC
    # If Fast SMA crosses over Slow SMA, Open Buy Trade
    if pricedata['dir'][pricedata.shape[0]-1] ==1:
        print("	  BUY SIGNAL!")
        if close_on_opposing_signal and countOpenTrades("S") > 0:
            print("	  Closing Sell Trade(s)...")
            exit("S")
        print("	  Opening Buy Trade...")
        enter("B")
    # If Fast SMA crosses under Slow SMA, Open Sell Trade
    if pricedata['dir'][pricedata.shape[0]-1] ==-1:
        print("	  SELL SIGNAL!")
        if close_on_opposing_signal and countOpenTrades("B") > 0:
            print("	  Closing Buy Trade(s)...")
            exit("B")
        print("	  Opening Sell Trade...")
        enter("S")
        
    print(str(dt.datetime.now()) + "	 " + timeframe + " Update Function Completed.\n")

In [97]:
get_latest_data()

Getting initial data...
Requesting Initial Price Data...
Initial Price Data Received...
initial price data
                     bidopen  bidclose  bidhigh   bidlow  askopen  askclose  \
date                                                                          
2019-07-09 13:40:00  1.12059   1.12026  1.12059  1.12026  1.12071   1.12038   
2019-07-09 13:45:00  1.12026   1.12036  1.12042  1.12010  1.12038   1.12048   
2019-07-09 13:50:00  1.12035   1.12051  1.12073  1.12034  1.12047   1.12062   
2019-07-09 13:55:00  1.12050   1.12062  1.12063  1.12034  1.12062   1.12076   
2019-07-09 14:00:00  1.12063   1.12051  1.12082  1.12050  1.12077   1.12065   

                     askhigh   asklow  tickqty  
date                                            
2019-07-09 13:40:00  1.12071  1.12038      850  
2019-07-09 13:45:00  1.12054  1.12023      984  
2019-07-09 13:50:00  1.12084  1.12046     1001  
2019-07-09 13:55:00  1.12076  1.12046     1247  
2019-07-09 14:00:00  1.12094  1.12062     111

AttributeError: 'NoneType' object has no attribute 'apply'